### signal.signal(signalnum, handler)
> 给定某个信号(signalnum, 有可能是系统给的，比如ctrl+c, 或者人为给定的，比如signal.alarm()), 就执行函数(handler, 可能用于清理之类的)

1. signalnum
 - SIGKILL(os.kill会发出这个信号)
 - SIGALRM(signal.alarm(s)会在s秒后发出这个信号)
 - SIGINT(ctrl + c时候系统会发出这个signal)
 - ...

2. handler(signalnum, frame)
 - SIG_IGN(This is another standard signal handler, which will simply ignore the given signal)
 - SIG_DFL(This is one of two standard signal handling options; it will simply perform the default function for the signal)

3. return the previous signal handler, for recovery, for example:  


```python
@contextmanager
def mask_sigint():
    """
    Returns:
        If called in main thread, returns a context where ``SIGINT`` is ignored, and yield True.
        Otherwise yield False.
    """
    if is_main_thread():
        # ignore SIGINT(KeyBoardInterrupt/Ctrl + c) and return handler for SIGINT(most likely SIG_DFL)
        sigint_handler = signal.signal(signal.SIGINT, signal.SIG_IGN)
        yield True
        # restore previous handler for SIGINT(SIG_DFL can be used if sigint_handler is not returned)
        signal.signal(signal.SIGINT, sigint_handler) 
    else:
        yield False

```
### signal.alarm(s)
> s秒后后发送SIGALRM, 并且取消之前任何的alarm, 当alarm=0也会取消之前的发送计划  

example:
```python
import errno
import os
import signal
import functools

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout) # 如果接收到SIGALRM信号, 执行_handle_timeout
            signal.alarm(seconds)                          # seconds秒后发送SIGALRM信号
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)                            # 函数已经执行完毕, 重置alarm。如果没在规定时间内执行完毕
            return result                                  # 则会raise TimeoutError

        return wrapper

    return decorator

@timeout(seconds=1)
def f(a, b):
    c = a +  b
    time.sleep(1.6)
    return c
f(1, 2)
```
similar example:
```python
import sys
import threading
from time import sleep
import _thread as thread

def cdquit(fn_name):
    # print to stderr, unbuffered in Python 2.
    print('{0} took too long'.format(fn_name), file=sys.stderr)
    sys.stderr.flush() # Python 3 stderr is likely buffered.
    thread.interrupt_main() # raises KeyboardInterrupt
    
def exit_after(s):
    '''
    use as decorator to exit process if 
    function takes longer than s seconds
    '''
    def outer(fn):
        def inner(*args, **kwargs):
            timer = threading.Timer(s, cdquit, args=[fn.__name__])
            timer.start()
            try:
                result = fn(*args, **kwargs)
            finally:
                timer.cancel()
            return result
        return inner
    return outer
    
@exit_after(1)
def a():
    time.sleep(1)
    print('a')

if __name__ == "__main__":
    a()
```